In [67]:
import numpy as np
import pandas as pd
import torch 
import requests
import time 

In [68]:
csv_data=pd.read_csv("../data/NTAD_Ferry_Routes_-1999537490590353801.csv")

In [69]:
ave_pax=csv_data["avg_daily_brd_pax"]
ave_veh=csv_data["avg_daily_brd_veh"]
starts=csv_data["seg_terminal1_id"]
ends=csv_data["seg_terminal2_id"]
start_times=csv_data["segment_season_start"]
end_times=csv_data["segment_season_end"]

In [70]:
superior_ids=[562,567, 308, 757, 914, 913, 909, 912, 882, 910,300, 911,283,289, 287, 296, ]
michigan_ids=[303,281,274,291,298, 288, 574, 564, 752, 570, 565, 568, 292, 694, 945, 908, 676, 944, 675,674, 804, 908,693,568]
huron_ids=[299, 282, 286, 284, 297]
erie_ids=[966,967,10,427,429,431,424,430,432]
superior_st_ids=[9099064, 9099090,9099044,9099018,9099004]
michigan_st_ids=[9087096, 9075080,9087088,9087069,9087068,9087072,9087077,9087057,9087044,9087031,9087023]
huron_st_ids=[9075065, 9075065, 9075035, 9075014]
erie_st_ids=[9063038, 9063028, 9063053, 9063063, 9063079, 9063085]
par_tmp= {
    "product": "daily_mean",         # 确保站点支持该产品
    "application": "UniversityOfGlasgow",
    "begin_date": "20230101",       # 时间范围不超过一年
    "end_date": "20231231",
    "station": "9087044",           # 确保站点 ID 有效
    "datum": "IGLD",
    "units": "metric",
    "time_zone": "gmt",
    "format": "json",
    "time_zone":"LST"
}
url = "https://api.tidesandcurrents.noaa.gov/api/prod/datagetter"
superior_level=[]
michigan_level=[]
huron_level=[]
erie_level=[]
for id in superior_st_ids:
    par_tmp["station"]=str(id) 
    response = requests.get(url, params=par_tmp)
    time.sleep(1)
    if response.status_code == 200:
        print(f"Succ:{id}")
        superior_level.append(response.json())
    else:
        print(f"请求失败，状态码：{response.status_code}")
        print(f"响应内容：{response.text}")
for id in michigan_st_ids:
    par_tmp["station"]=str(id) 
    response = requests.get(url, params=par_tmp)
    time.sleep(1)
    if response.status_code == 200:
        print(f"Succ:{id}")
        michigan_level.append(response.json())
    else:
        print(f"请求失败，状态码：{response.status_code}")
        print(f"响应内容：{response.text}") 
for id in huron_st_ids:
    par_tmp["station"]=str(id) 
    response = requests.get(url, params=par_tmp)
    time.sleep(1)
    if response.status_code == 200:
        print(f"Succ:{id}")
        huron_level.append(response.json())
    else:
        print(f"请求失败，状态码：{response.status_code}")
        print(f"响应内容：{response.text}")  
for id in erie_st_ids:
    par_tmp["station"]=str(id) 
    response = requests.get(url, params=par_tmp)
    time.sleep(1)
    if response.status_code == 200:
        print(f"Succ:{id}")
        erie_level.append(response.json())
    else:
        print(f"请求失败，状态码：{response.status_code}")
        print(f"响应内容：{response.text}")

Succ:9099064
Succ:9099090
Succ:9099044
Succ:9099018
Succ:9099004
Succ:9087096
Succ:9075080
Succ:9087088
Succ:9087069
Succ:9087068
Succ:9087072
Succ:9087077
Succ:9087057
Succ:9087044
Succ:9087031
Succ:9087023
Succ:9075065
Succ:9075065
Succ:9075035
Succ:9075014
Succ:9063038
Succ:9063028
Succ:9063053
Succ:9063063
Succ:9063079
Succ:9063085


In [71]:
avg_msl_superior={}
avg_msl_michigan={}
avg_msl_huron={}
avg_msl_erie={}
for data_r in superior_level:
    for data in data_r["data"]:
        if not data["t"] in avg_msl_superior.keys():
            avg_msl_superior[data["t"]]=[]
        avg_msl_superior[data["t"]].append(float(data["v"]))
for data_r in michigan_level:
    if not "data" in data_r.keys():
        continue
    for data in data_r["data"]:
        if not data["t"] in avg_msl_michigan.keys():
            avg_msl_michigan[data["t"]]=[]
        avg_msl_michigan[data["t"]].append(float(data["v"]))
for data_r in huron_level:
    if not "data" in data_r.keys():
        continue
    for data in data_r["data"]:
        if not data["t"] in avg_msl_huron.keys():
            avg_msl_huron[data["t"]]=[]
        avg_msl_huron[data["t"]].append(float(data["v"]))
for data_r in erie_level:
    if not "data" in data_r.keys():
        continue
    for data in data_r["data"]:
        if not data["t"] in avg_msl_erie.keys():
            avg_msl_erie[data["t"]]=[]
        avg_msl_erie[data["t"]].append(float(data["v"]))

In [72]:
for key in avg_msl_superior.keys():
    avg_msl_superior[key]=np.mean(avg_msl_superior[key])
for key in avg_msl_michigan.keys():
    avg_msl_michigan[key]=np.mean(avg_msl_michigan[key])
for key in avg_msl_erie.keys():
    avg_msl_erie[key]=np.mean(avg_msl_erie[key])
for key in avg_msl_huron.keys():
    avg_msl_huron[key]=np.mean(avg_msl_huron[key])

In [73]:
ship_data=[]
def construct(x:str):
    mm,dd=x.split("/")
    return f"2023-{mm}-{dd} 00:00"
demo_sup=[]
for indx, (start,end) in enumerate(zip(starts, ends)):
    try:
        if start in superior_ids and end in superior_ids:
            # print(f"st:{start_times[indx]},en:{end_times[indx]}")
            if start_times[indx]=="" or end_times[indx]=="" or end_times[indx]==float("nan") or start_times[indx]==float("nan"):
                continue
            
            sts=construct(start_times[indx])
            ens=construct(end_times[indx])
            print(f"{avg_msl_superior[sts]},{avg_msl_superior[ens]}")
            demo_sup.append((ave_pax[indx]+20*ave_veh[indx],(avg_msl_superior[sts]+avg_msl_superior[ens])/2))
            # 2023-02-02 00:00
        elif start in michigan_ids and end in michigan_ids:
            if start_times[indx]=="" or end_times[indx]=="":
                continue
        elif start in huron_ids and end in huron_ids:
            if start_times[indx]=="" or end_times[indx]=="":
                continue
        elif start in erie_ids and end in erie_ids:
            if start_times[indx]=="" or end_times[indx]=="":
                continue
    except:
        pass

183.66619999999998,183.58780000000002
183.66619999999998,183.58780000000002
183.66619999999998,183.58780000000002
183.66619999999998,183.58780000000002
183.66619999999998,183.58780000000002
183.66619999999998,183.58780000000002
183.66619999999998,183.58780000000002
183.66619999999998,183.58780000000002
183.66619999999998,183.58780000000002
183.66619999999998,183.58780000000002
183.66619999999998,183.58780000000002
183.66619999999998,183.58780000000002
183.66619999999998,183.58780000000002
183.66619999999998,183.58780000000002
183.66619999999998,183.58780000000002
183.66619999999998,183.58780000000002
183.66619999999998,183.58780000000002
183.66619999999998,183.58780000000002
183.66619999999998,183.58780000000002
183.66619999999998,183.58780000000002
183.66619999999998,183.58780000000002
183.57959999999997,183.58780000000002
183.66619999999998,183.58780000000002
183.66619999999998,183.58780000000002
183.66619999999998,183.58780000000002
183.66619999999998,183.58780000000002
183.66619999

In [74]:
from plotly import graph_objects as go 
fig=go.Figure()
fig.add_trace(go.Scatter(y=[i[0] for i in demo_sup], x=[i[1] for i in demo_sup], mode="markers"))
fig.show()

In [75]:
bird=[(2022,391),(2021,391),(2021,379),(2020,373),(2019,373),(2018,377),(2017,379),(2016,380),(2015,370),(2014,369),(2013,368),(2012,362),(2011,371),(2010, 363),(2009,352),(2008,362),()]
# 2018